In [1]:
import dask.dataframe as dd
import polars as pl

In [2]:
df_dd = dd.read_csv(
    './dados/*.csv',
    assume_missing=True,
    sep=',',
    encoding='latin1',
    dtype={
        'RELIGIAO': 'object',
        'VINCULACAO_ANO': 'object'
    }
)

In [3]:
df_dd.head()

,ANO_NASCIMENTO,PESO,ALTURA,CABECA,CALCADO,CINTURA,RELIGIAO,MUN_NASCIMENTO,UF_NASCIMENTO,PAIS_NASCIMENTO,...,ESCOLARIDADE,VINCULACAO_ANO,DISPENSA,ZONA_RESIDENCIAL,MUN_RESIDENCIA,UF_RESIDENCIA,PAIS_RESIDENCIA,JSM,MUN_JSM,UF_JSM
0,1989.0,95.0,181.0,57.0,44.0,98.0,<NA>,DESCALVADO,SP,BRASIL,...,Ensino Médio Completo,2007,Sem dispensa,Urbana,PIRASSUNUNGA,SP,BRASIL,5/93 - PIRASSUNUNGA,PIRASSUNUNGA,SP
1,1989.0,64.0,173.0,57.0,41.0,76.0,<NA>,CATANDUVA,SP,BRASIL,...,Ensino Superior Completo,2007,Sem dispensa,Urbana,SAO PAULO,SP,BRASIL,4/94 - SAO PAULO (BUTANTA),SAO PAULO,SP
2,1989.0,82.0,171.0,58.0,41.0,94.0,<NA>,CATANDUVA,SP,BRASIL,...,Ensino Médio Completo,2007,Sem dispensa,Urbana,CATANDUVA,SP,BRASIL,5/27 - CATANDUVA,CATANDUVA,SP
3,1989.0,NaN,NaN,NaN,NaN,NaN,<NA>,SANTO ANDRE,SP,BRASIL,...,Ensino Médio Completo,2007,Sem dispensa,Urbana,PIRAJUI,SP,BRASIL,6/127 - PIRAJUI,PIRAJUI,SP
4,1989.0,NaN,NaN,NaN,NaN,NaN,<NA>,PIRAJUI,SP,BRASIL,...,3ª Série do Ensino Médio,2007,Sem dispensa,Urbana,PIRAJUI,SP,BRASIL,6/127 - PIRAJUI,PIRAJUI,SP


In [4]:
df_dd.dtypes

ANO_NASCIMENTO              float64
PESO                        float64
ALTURA                      float64
CABECA                      float64
CALCADO                     float64
CINTURA                     float64
RELIGIAO            string[pyarrow]
MUN_NASCIMENTO      string[pyarrow]
UF_NASCIMENTO       string[pyarrow]
PAIS_NASCIMENTO     string[pyarrow]
ESTADO_CIVIL        string[pyarrow]
SEXO                string[pyarrow]
ESCOLARIDADE        string[pyarrow]
VINCULACAO_ANO      string[pyarrow]
DISPENSA            string[pyarrow]
ZONA_RESIDENCIAL    string[pyarrow]
MUN_RESIDENCIA      string[pyarrow]
UF_RESIDENCIA       string[pyarrow]
PAIS_RESIDENCIA     string[pyarrow]
JSM                 string[pyarrow]
MUN_JSM             string[pyarrow]
UF_JSM              string[pyarrow]
dtype: object

In [5]:
#df_dd['VINCULACAO_ANO'].compute().value_counts()

In [6]:
df_dd = (df_dd.loc[~df_dd['VINCULACAO_ANO'].isin(['Com dispensa', 'Sem dispensa'])]
    .astype({
        'VINCULACAO_ANO': 'int16',

    }).drop(columns=[
        'JSM', 
        'MUN_JSM', 
        'UF_JSM',
        'CABECA',
        'CALCADO',
        'CINTURA'
    ]).fillna({
        'RELIGIAO': 'Ignorado' 
    })
)

In [7]:
df_dd.head()

,ANO_NASCIMENTO,PESO,ALTURA,RELIGIAO,MUN_NASCIMENTO,UF_NASCIMENTO,PAIS_NASCIMENTO,ESTADO_CIVIL,SEXO,ESCOLARIDADE,VINCULACAO_ANO,DISPENSA,ZONA_RESIDENCIAL,MUN_RESIDENCIA,UF_RESIDENCIA,PAIS_RESIDENCIA
0,1989.0,95.0,181.0,Ignorado,DESCALVADO,SP,BRASIL,Solteiro,M,Ensino Médio Completo,2007,Sem dispensa,Urbana,PIRASSUNUNGA,SP,BRASIL
1,1989.0,64.0,173.0,Ignorado,CATANDUVA,SP,BRASIL,Casado,M,Ensino Superior Completo,2007,Sem dispensa,Urbana,SAO PAULO,SP,BRASIL
2,1989.0,82.0,171.0,Ignorado,CATANDUVA,SP,BRASIL,Solteiro,M,Ensino Médio Completo,2007,Sem dispensa,Urbana,CATANDUVA,SP,BRASIL
3,1989.0,NaN,NaN,Ignorado,SANTO ANDRE,SP,BRASIL,Solteiro,M,Ensino Médio Completo,2007,Sem dispensa,Urbana,PIRAJUI,SP,BRASIL
4,1989.0,NaN,NaN,Ignorado,PIRAJUI,SP,BRASIL,Solteiro,M,3ª Série do Ensino Médio,2007,Sem dispensa,Urbana,PIRAJUI,SP,BRASIL


In [8]:
df_dd.dtypes

ANO_NASCIMENTO              float64
PESO                        float64
ALTURA                      float64
RELIGIAO            string[pyarrow]
MUN_NASCIMENTO      string[pyarrow]
UF_NASCIMENTO       string[pyarrow]
PAIS_NASCIMENTO     string[pyarrow]
ESTADO_CIVIL        string[pyarrow]
SEXO                string[pyarrow]
ESCOLARIDADE        string[pyarrow]
VINCULACAO_ANO                int16
DISPENSA            string[pyarrow]
ZONA_RESIDENCIAL    string[pyarrow]
MUN_RESIDENCIA      string[pyarrow]
UF_RESIDENCIA       string[pyarrow]
PAIS_RESIDENCIA     string[pyarrow]
dtype: object

In [9]:
#int_cols = [col for col in df_dd.select_dtypes(include=['int16']).columns if col != 'VINCULACAO_ANO']

In [10]:
#nos = df_dd['VINCULACAO_ANO'].compute().unique().tolist()

In [11]:
dd.to_parquet(
    df_dd,
    './dados_parquet/',
    engine='pyarrow',
    compression='snappy',
    write_index=False
)

KeyError: "['ANO_NASCIMENTO', 'RELIGIAO'] not in index"